In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import KNNImputer, SimpleImputer

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from sklearn.preprocessing import StandardScaler , OneHotEncoder, MinMaxScaler
# import category_encoders as ce

import random
import seaborn as sns


In [2]:
df = sns.load_dataset('mpg')

In [3]:
# sns.get_dataset_names()

In [4]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [5]:
df.origin.unique()

array(['usa', 'japan', 'europe'], dtype=object)

In [6]:
df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model_year', 'origin', 'name'],
      dtype='object')

Общая структура данных в наборе "mpg" следующая:

- origin: производитель автомобиля.
- name: модель автомобиля.
- displacement: объем двигателя в литрах.
- model_year: год выпуска автомобиля.
- cylinders: количество цилиндров в двигателе.
- mpg: расход топлива в городе (мили на галлон).

In [7]:
# типы столбцов
id = 'name' # столбец ID
target = 'mpg' # столбец с целевой переменной
features_int = [col for col in df.select_dtypes(exclude='object').columns if col != id and col !=target]
features_cat = [col for col in df.select_dtypes(include='object').columns if col != id and col !=target]

# EDA

In [8]:
df.shape

(398, 9)

In [9]:
df.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64

есть пропуски horsepower

In [10]:
df.describe(include='all')

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000,398,398
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,305
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,usa,ford pinto
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249,6
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050,NaN,NaN
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627,NaN,NaN
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,NaN,NaN
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000,NaN,NaN
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000,NaN,NaN
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000,NaN,NaN


Пропуски заполним самым частым значением с помощью SimpleImputer.

Номинальный признак origin закодируем с помощью OneHotEncoder.

Количественные признаки масштабируем для моделей, для которых это необходимо.

# Делим данные

In [11]:
# df.dropna(inplace=True)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop([target, id], axis=1), df[target], test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((318, 7), (80, 7))

# Моделирование

## подготовка pipeline

In [13]:
features_int

['cylinders',
 'displacement',
 'horsepower',
 'weight',
 'acceleration',
 'model_year']

In [14]:
features_cat

['origin']

In [15]:
# SimpleImputer + OHE
ohe_pipe = Pipeline(
    [
        (
            'imputer',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ohe',
            OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
        )
    ]
)

In [16]:
num_pipe = Pipeline(
    [
        (
            'imputer',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'scaler',
            StandardScaler()
        )
    ]
)

In [17]:
data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, features_cat),
        ('num', num_pipe, features_int),
    ],
    remainder='passthrough'
)

In [18]:
# итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('models', DecisionTreeRegressor(random_state=12345))
    ]
)

In [19]:
pipe_final

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['origin']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['cylinders', 'displacement',
                                                   'horsepower', 'weight',
                                                   'acceleration',
                                                   'model_year'])])),
                ('models', DecisionTreeRegressor(random_state=12345))])

In [20]:
# обучаем модель на тренировочной выборке
pipe_final.fit(X_train, y_train)

# выводим предсказанные значения тренировочной выборки
y_train_pred = pipe_final.predict(X_train)
y_test_pred = pipe_final.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
X_train_p = pd.DataFrame(
    data_preprocessor.fit_transform(X_train),
    columns=data_preprocessor.get_feature_names_out()
)

X_test_p = pd.DataFrame(
    data_preprocessor.transform(X_test),
    columns=data_preprocessor.get_feature_names_out()
)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## GridSearchCV

In [22]:
param_grid = {
    'preprocessor__num__imputer':[SimpleImputer(), KNNImputer() ],
    'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), 'passthrough'] ,
    'models' : [LinearRegression(), DecisionTreeRegressor(), SGDRegressor()]
}

In [23]:
pipe_final

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['origin']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['cylinders', 'displacement',
                                                   'horsepower', 'weight',
                                                   'acceleration',
                                                   'model_year'])])),
                ('models', DecisionTreeRegressor(random_state=12345))])

In [24]:
%%time
grid = GridSearchCV(
    pipe_final,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs = -1
)
grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         ['origin']),
                                                                        ('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent'...
                                                                          'acceleration',
                                                                          'model_year'])])),
                                       ('models',
                                        DecisionTreeRegressor(random_state=12345))]),
             n_jobs=-1,
             param_grid={'models': [LinearRegression(), DecisionTreeRegressor(),
                                    SGDRegressor()],
                         'preprocessor__num__imputer': [SimpleImputer(),
                                                        KNNImputer()],
                         'preprocessor__num__scaler': [StandardScaler(),
                                                       MinMaxScaler(),
                                                       'passthrough']},
             scoring='neg_mean_absolute_error')

In [25]:
df_cv_results = pd.DataFrame(grid.cv_results_)

In [26]:
# df_cv_results

In [27]:

print('Лучшая модель и её параметры:\n\n', grid.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', grid.best_score_)


Лучшая модель и её параметры:

 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['origin']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                           

## RandomSearch

In [28]:
grid_rs = RandomizedSearchCV(
    pipe_final,
    param_distributions=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs = -1
)
grid_rs.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('ohe',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('ohe',
                                                                                                OneHotEncoder(drop='first',
                                                                                                              handle_unknown='ignore',
                                                                                                              sparse=False))]),
                                                                               ['origin']),
                                                                              ('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='most_fre...
                                                                                'acceleration',
                                                                                'model_year'])])),
                                             ('models',
                                              DecisionTreeRegressor(random_state=12345))]),
                   n_jobs=-1,
                   param_distributions={'models': [LinearRegression(),
                                                   DecisionTreeRegressor(),
                                                   SGDRegressor()],
                                        'preprocessor__num__imputer': [SimpleImputer(),
                                                                       KNNImputer()],
                                        'preprocessor__num__scaler': [StandardScaler(),
                                                                      MinMaxScaler(),
                                                                      'passthrough']},
                   scoring='neg_mean_absolute_error')

In [29]:
df_cv_results = pd.DataFrame(grid_rs.cv_results_)

In [30]:
df_cv_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessor__num__scaler,param_preprocessor__num__imputer,param_models,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.113382,0.017205,0.073529,0.033571,passthrough,KNNImputer(),DecisionTreeRegressor(),"{'preprocessor__num__scaler': 'passthrough', '...",-2.753125e+00,-2.176563e+00,-2.629688e+00,-2.663492e+00,-2.690476e+00,-2.582669e+00,2.070475e-01,1
1,0.117301,0.016416,0.064740,0.017564,passthrough,KNNImputer(),LinearRegression(),"{'preprocessor__num__scaler': 'passthrough', '...",-2.592253e+00,-2.406459e+00,-2.702535e+00,-2.766736e+00,-2.862573e+00,-2.666111e+00,1.567955e-01,5
2,0.114599,0.011451,0.058298,0.016466,MinMaxScaler(),KNNImputer(),DecisionTreeRegressor(),"{'preprocessor__num__scaler': MinMaxScaler(), ...",-2.954688e+00,-2.235938e+00,-2.679688e+00,-2.679365e+00,-2.728571e+00,-2.655650e+00,2.332551e-01,4
3,0.082466,0.028774,0.045677,0.016165,StandardScaler(),KNNImputer(),LinearRegression(),{'preprocessor__num__scaler': StandardScaler()...,-2.592253e+00,-2.406459e+00,-2.702535e+00,-2.766736e+00,-2.862573e+00,-2.666111e+00,1.567955e-01,6
4,0.138055,0.035127,0.059387,0.016904,passthrough,SimpleImputer(),SGDRegressor(),"{'preprocessor__num__scaler': 'passthrough', '...",-3.157082e+15,-3.922486e+15,-3.179387e+15,-2.050876e+15,-6.031957e+14,-2.582605e+15,1.156049e+15,10


In [31]:

print('Лучшая модель и её параметры:\n\n', grid_rs.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', grid_rs.best_score_)


Лучшая модель и её параметры:

 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['origin']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                           

## HalvingGridSearchCV

HalvingGridSearchCV сочетает Grid Search и Successive Halving, чтобы улучшить эффективность поиска оптимальных гиперпараметров. Он начинает с оценки всех комбинаций гиперпараметров с небольшим количеством ресурсов (например, малое количество итераций или маленький подвыборку данных), затем отбирает лучшие и оценивает их с большим количеством ресурсов.

Successive Halving (Постепенное сокращение): Идея метода заключается в том, что на начальных этапах рассматривается много комбинаций гиперпараметров, но каждая из них оценивается на небольшом числе обучающих данных. На каждом следующем этапе число комбинаций уменьшается (удаляя наименее эффективные), но увеличивается количество данных, используемых для оценки оставшихся комбинаций.

Grid Search (Перебор сетки): Как и в обычном GridSearchCV, HalvingGridSearchCV ищет лучшие гиперпараметры по сетке возможных значений.

Преимущества
Эффективность: Значительно уменьшает время обучения по сравнению с обычным GridSearchCV, особенно на больших датасетах.
Гибкость: Позволяет настроить параметры для баланса между временем обучения и качеством модели.

Основные параметры
- estimator: модель, для которой необходимо провести поиск гиперпараметров.
- param_grid: сетка гиперпараметров для перебора.
factor: множитель, определяющий, насколько быстро сокращается число комбинаций гиперпараметров (по умолчанию 3).
- resource: Этот параметр указывает, какой ресурс будет увеличиваться на каждом этапе. Обычно это количество образцов (n_samples), но может быть и количество признаков (n_features). По умолчанию используется n_samples.
- min_resources: Минимальное количество ресурсов, которое будет использовано на первом этапе. Этот параметр задает нижний предел количества данных, которые будут использоваться для оценки на начальном этапе. Если этот параметр не задан, HalvingGridSearchCV автоматически определяет его.
- max_resources: Максимальное количество ресурсов, которое может быть использовано. Если не указано, будет использовано количество данных, равное размеру всего набора данных.
- factor: Множитель, определяющий, насколько быстро сокращается число комбинаций гиперпараметров и увеличивается количество используемых данных на каждом этапе. Например, при factor=2 количество данных будет удваиваться на каждом этапе.
- cv: количество фолдов для кросс-валидации.

In [32]:
grid_hs = HalvingGridSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs = -1
)
grid_hs.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(remainder='passthrough',
                                                                 transformers=[('ohe',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='most_frequent')),
                                                                                                ('ohe',
                                                                                                 OneHotEncoder(drop='first',
                                                                                                               handle_unknown='ignore',
                                                                                                               sparse=False))]),
                                                                                ['origin']),
                                                                               ('num',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='most_frequen...
                                                                                 'acceleration',
                                                                                 'model_year'])])),
                                              ('models',
                                               DecisionTreeRegressor(random_state=12345))]),
                    n_jobs=-1,
                    param_grid={'models': [LinearRegression(),
                                           DecisionTreeRegressor(),
                                           SGDRegressor()],
                                'preprocessor__num__imputer': [SimpleImputer(),
                                                               KNNImputer()],
                                'preprocessor__num__scaler': [StandardScaler(),
                                                              MinMaxScaler(),
                                                              'passthrough']},
                    scoring='neg_mean_absolute_error')

In [33]:
df_cv_results = pd.DataFrame(grid_hs.cv_results_)

## Optuna

Optuna — это библиотека на Python для автоматизированной настройки гиперпараметров. Основной метод, применяемый в Optuna, это Байесовская оптимизация, в частности, с использованием алгоритма Tree-structured Parzen Estimator (TPE).


Tree-structured Parzen Estimator (TPE) — это метод байесовской оптимизации, который используется для поиска оптимальных гиперпараметров в задачах машинного обучения. TPE является одним из основных алгоритмов, используемых в библиотеке Optuna. В отличие от методов, основанных на Гауссовских процессах, TPE использует непараметрические методы оценки плотности для моделирования распределения гиперпараметров.

Основные концепции TPE
1. TPE делит пространство гиперпараметров на две части:

- Набор лучших значений: значения гиперпараметров, которые дают наилучшие результаты функции цели.
- Остальные значения: все остальные значения гиперпараметров.

2. Оценка плотности вероятности
TPE строит два оценщика плотности:
 - l(x): плотность вероятности гиперпараметров для лучших значений функции цели $y^*<y$

 - g(x): плотность вероятности гиперпараметров для остальных значений $y^* >= y$
 Здесь $y^*$ — это пороговое значение, которое разделяет лучшие и остальные значения. Обычно  $y^*$
  выбирается как некоторое квантиль распределения значений функции цели, например, 10%-й квантиль.

3. 3. Функция полезности
TPE максимизирует отношение плотностей $ \frac{l(x)}{g(x)}$
 , чтобы найти такие значения гиперпараметров x, которые с наибольшей вероятностью улучшат значение функции цели.


In [34]:
!pip install optuna -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.7 MB/s eta 0:00:00


In [35]:
!pip install optuna-integration -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 1.4 MB/s eta 0:00:00


In [36]:
import optuna

In [37]:
from optuna.integration import OptunaSearchCV
from optuna import distributions

In [38]:
# инициализируем модель дерева решений
model = DecisionTreeRegressor(random_state=123)

# создаём словарь с гиперпараметрами
parameters = {
    'max_depth': distributions.IntDistribution(3, 17),
    'min_samples_split': distributions.IntDistribution(9, 10)
}

In [39]:
model_l = Lasso(random_state=123)
parameters_l = {'alpha': distributions.IntDistribution(3, 10)}

In [40]:
def optuna_func(model, parameters):
  opt_search = OptunaSearchCV(
      model,
      parameters,
      cv=5,
      n_trials=10,
      random_state=1234,
      verbose=-10
  )

  # запускаем поиск гиперпараметров
  opt_search.fit(X_train_p, y_train)
  return opt_search.best_params_, opt_search.best_estimator_

In [41]:
best_params_tree, best_estimator_tree = optuna_func(model, parameters)

<ipython-input-40-f7f053cb1956>:2: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  opt_search = OptunaSearchCV(
[I 2024-07-03 15:39:51,763] A new study created in memory with name: no-name-ca57d0e3-63df-43c5-9135-ed73c957cfc7
[I 2024-07-03 15:39:51,805] Trial 0 finished with value: 0.7984142531359933 and parameters: {'max_depth': 14, 'min_samples_split': 9}. Best is trial 0 with value: 0.7984142531359933.
[I 2024-07-03 15:39:51,838] Trial 1 finished with value: 0.7581564501629479 and parameters: {'max_depth': 3, 'min_samples_split': 10}. Best is trial 0 with value: 0.7984142531359933.
[I 2024-07-03 15:39:51,886] Trial 2 finished with value: 0.7946450518036884 and parameters: {'max_depth': 7, 'min_samples_split': 9}. Best is trial 0 with value: 0.7984142531359933.
[I 2024-07-03 15:39:51,920] Trial 3 finished with value: 0.7987756923093049 and parameters: {'max_depth': 11, 'min_samples_split': 10}. Best is trial 3 wi

In [42]:
pred_test = best_estimator_tree.predict(X_test_p)


In [43]:
from sklearn.metrics import mean_squared_error

In [44]:
mse_dtr = mean_squared_error(y_test, pred_test)
mse_dtr

10.774043984237213

In [45]:
best_params_l, best_estimator_l = optuna_func(model_l, parameters_l)

<ipython-input-40-f7f053cb1956>:2: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  opt_search = OptunaSearchCV(
[I 2024-07-03 15:39:52,333] A new study created in memory with name: no-name-6932d39a-1144-44ed-82b2-ea0c33bfabf4
[I 2024-07-03 15:39:52,405] Trial 0 finished with value: -0.01931215576997376 and parameters: {'alpha': 9}. Best is trial 0 with value: -0.01931215576997376.
[I 2024-07-03 15:39:52,470] Trial 1 finished with value: 0.572924477920204 and parameters: {'alpha': 3}. Best is trial 1 with value: 0.572924477920204.
[I 2024-07-03 15:39:52,529] Trial 2 finished with value: 0.572924477920204 and parameters: {'alpha': 3}. Best is trial 1 with value: 0.572924477920204.
[I 2024-07-03 15:39:52,604] Trial 3 finished with value: -0.01931215576997376 and parameters: {'alpha': 9}. Best is trial 1 with value: 0.572924477920204.
[I 2024-07-03 15:39:52,684] Trial 4 finished with value: 0.2698428078515403 and param

In [46]:
pred_test_l = best_estimator_l.predict(X_test_p)

mse_l = mean_squared_error(y_test, pred_test_l)
mse_l

18.63578973298567

In [47]:
models_dict = {'tree': [DecisionTreeRegressor(random_state=123), {
                                                                'max_depth': distributions.IntDistribution(3, 10),
                                                                'min_samples_split': distributions.IntDistribution(9, 10)
                                                            }  ],
               'lasso':[Lasso(random_state=123), {'alpha': distributions.IntDistribution(3, 10)}]
               }

In [48]:
result = {}
for key, value in models_dict.items():
  best_params, best_estimator = optuna_func(value[0], value[1])
  pred_test = best_estimator.predict(X_test_p)
  mse = mean_squared_error(y_test, pred_test)
  result[key] = mse


<ipython-input-40-f7f053cb1956>:2: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  opt_search = OptunaSearchCV(
[I 2024-07-03 15:39:53,222] A new study created in memory with name: no-name-4449e9bd-aa04-4a3d-b37b-3202632f1b8e
[I 2024-07-03 15:39:53,411] Trial 0 finished with value: 0.7954545612148088 and parameters: {'max_depth': 9, 'min_samples_split': 9}. Best is trial 0 with value: 0.7954545612148088.
[I 2024-07-03 15:39:53,463] Trial 1 finished with value: 0.7581564501629479 and parameters: {'max_depth': 3, 'min_samples_split': 10}. Best is trial 0 with value: 0.7954545612148088.
[I 2024-07-03 15:39:53,513] Trial 2 finished with value: 0.793090558135407 and parameters: {'max_depth': 5, 'min_samples_split': 9}. Best is trial 0 with value: 0.7954545612148088.
[I 2024-07-03 15:39:53,603] Trial 3 finished with value: 0.7961771140224546 and parameters: {'max_depth': 7, 'min_samples_split': 10}. Best is trial 3 with 

In [49]:
result

{'tree': 10.78942358975419, 'lasso': 18.63578973298567}